# **Average Height Exercise (Core)**
**Student**: Michael McCann <br>
**Date**: 06 FEB 2022

In [2]:
## Load Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
## Import libraries
import numpy as np
import pandas as pd

In [4]:
## Load Data: 
filepath = "/content/drive/MyDrive/Data/athleteEventsNoPersonal.csv" 
oly_df = pd.read_csv(filepath)

# Explore the Data


## (1A) How many rows? and (1B) How many columns?
The dataframe is made up of 12 columns and 40616 rows. <br>
code follows

In [5]:
display(oly_df.head())
oly_df.info()

,ID,Age,Height,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,5,21.0,185.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN
1,5,21.0,185.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,"Speed Skating Women's 1,000 metres",NaN
2,5,25.0,185.0,Netherlands,NED,1992 Winter,1992,Winter,Albertville,Speed Skating,Speed Skating Women's 500 metres,NaN
3,5,25.0,185.0,Netherlands,NED,1992 Winter,1992,Winter,Albertville,Speed Skating,"Speed Skating Women's 1,000 metres",NaN
4,5,27.0,185.0,Netherlands,NED,1994 Winter,1994,Winter,Lillehammer,Speed Skating,Speed Skating Women's 500 metres,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40616 entries, 0 to 40615
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      40616 non-null  int64  
 1   Age     39143 non-null  float64
 2   Height  31615 non-null  float64
 3   Team    40616 non-null  object 
 4   NOC     40616 non-null  object 
 5   Games   40616 non-null  object 
 6   Year    40616 non-null  int64  
 7   Season  40616 non-null  object 
 8   City    40616 non-null  object 
 9   Sport   40616 non-null  object 
 10  Event   40616 non-null  object 
 11  Medal   5917 non-null   object 
dtypes: float64(2), int64(2), object(8)
memory usage: 3.7+ MB


## (1C) Which columns have missing values? (Why do think there are there so many null values in the Medals column?)
Age, height, and medal have missing values.
- replace missing values in age and height with average values of olympians competing in the same sport or event in the same year. 
- replace missing medal values with 'no medal' to indicate they did not medal in that years games. 

In [6]:
oly_df[oly_df.duplicated(keep = False)]
oly_df.drop_duplicates(inplace = True)
oly_df

,ID,Age,Height,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,5,21.0,185.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN
1,5,21.0,185.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,"Speed Skating Women's 1,000 metres",NaN
2,5,25.0,185.0,Netherlands,NED,1992 Winter,1992,Winter,Albertville,Speed Skating,Speed Skating Women's 500 metres,NaN
3,5,25.0,185.0,Netherlands,NED,1992 Winter,1992,Winter,Albertville,Speed Skating,"Speed Skating Women's 1,000 metres",NaN
4,5,27.0,185.0,Netherlands,NED,1994 Winter,1994,Winter,Lillehammer,Speed Skating,Speed Skating Women's 500 metres,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
40611,135567,28.0,183.0,Russia,RUS,2004 Summer,2004,Summer,Athina,Rowing,Rowing Men's Lightweight Coxless Fours,NaN
40612,135570,27.0,176.0,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Individual",NaN
40613,135570,27.0,176.0,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Team",NaN
40614,135571,30.0,185.0,Poland,POL,1998 Winter,1998,Winter,Nagano,Bobsleigh,Bobsleigh Men's Four,NaN


In [7]:
display(oly_df.isna().sum())

#Replace NaNs using the average of other athletes competing in the same Event, Sport, or Season depending on available data.
oly_df['Age'] = oly_df['Age'].fillna(oly_df.groupby(['Year', 'Event'])['Age'].transform('mean'))
oly_df['Age'] = oly_df['Age'].fillna(oly_df.groupby(['Year', 'Sport'])['Age'].transform('mean'))
oly_df['Age'] = oly_df['Age'].fillna(oly_df.groupby(['Year', 'Season'])['Age'].transform('mean'))

#Replace NaNs using the average of other athletes competing in the same Event, Sport, or Season depending on available data.
oly_df['Height'] = oly_df['Height'].fillna(oly_df.groupby(['Year', 'Event'])['Height'].transform('mean'))
oly_df['Height'] = oly_df['Height'].fillna(oly_df.groupby(['Year', 'Sport'])['Height'].transform('mean'))
oly_df['Height'] = oly_df['Height'].fillna(oly_df.groupby(['Year', 'Season'])['Height'].transform('mean'))

# Replace Medal NaNs with No Medal
oly_df['Medal'].fillna('No Medal', inplace = True)

ID            0
Age        1446
Height     8796
Team          0
NOC           0
Games         0
Year          0
Season        0
City          0
Sport         0
Event         0
Medal     34493
dtype: int64

## (1D) How many entries correspond to the city of London?
- 472 entries from the 1908 Summer games in London
- 940 entries from the 1948 Summer games in London
- 1935 entries from the 2012 Summer games in London
- 3347 in total. 

In [8]:
oly_df[oly_df['City'] == 'London'].groupby(['Games'])['City'].value_counts()

Games        City  
1908 Summer  London     472
1948 Summer  London     940
2012 Summer  London    1935
Name: City, dtype: int64

## (1E) What age is the youngest athlete in our sample data?  Hint: use min()

- Three Olympians (from this dataset) have competed at the age of 11 in the 1924 Winter Games, 1932 Winter Games, and the 1968 Winter Games.
- One 74 year old competed in the 1924 Olympics in the mixed sculpting competition.

In [9]:
display(oly_df[oly_df['Age'] == oly_df['Age'].min()])
print("\n")
oly_df[oly_df['Age'] == oly_df['Age'].max()]

,ID,Age,Height,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
6480,22411,11.0,152.0,Great Britain,GBR,1932 Winter,1932,Winter,Lake Placid,Figure Skating,Figure Skating Women's Singles,No Medal
14025,47618,11.0,155.0,Norway,NOR,1924 Winter,1924,Winter,Chamonix,Figure Skating,Figure Skating Women's Singles,No Medal
15103,51268,11.0,151.0,Romania,ROU,1968 Winter,1968,Winter,Grenoble,Figure Skating,Figure Skating Women's Singles,No Medal


,ID,Age,Height,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
30401,101272,74.0,175.521762,France,FRA,1924 Summer,1924,Summer,Paris,Art Competitions,Art Competitions Mixed Sculpturing,No Medal


# Create Filters to Find Out:

## (2A) How many athletes who participated in the Sport "Tug-Of-War" were from Team "Sweden" in our sample dataset?
- 6 athlets from the 1906 and 1908 olympics.

In [10]:
tow_filter = oly_df['Sport'] == 'Tug-Of-War'
swe_filter = oly_df['Team'] == 'Sweden'

oly_df[tow_filter & swe_filter]

,ID,Age,Height,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
10023,34205,22.0,180.008757,Sweden,SWE,1908 Summer,1908,Summer,London,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,No Medal
12681,43105,23.0,184.666667,Sweden,SWE,1906 Summer,1906,Summer,Athina,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Bronze
13015,44087,29.0,182.000000,Sweden,SWE,1906 Summer,1906,Summer,Athina,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Bronze
16250,55169,23.0,180.008757,Sweden,SWE,1908 Summer,1908,Summer,London,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,No Medal
19048,64266,21.0,180.008757,Sweden,SWE,1908 Summer,1908,Summer,London,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,No Medal
20550,68647,26.0,190.000000,Sweden,SWE,1906 Summer,1906,Summer,Athina,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Bronze


## (2B) How many Gold medals were awarded to athletes who were over 40 years old?
- 69 gold medals were awarded to athletes over 40.

In [11]:
gold_filter = oly_df['Medal'] == 'Gold'
o40_filter = oly_df['Age'] > 40.0

oly_df[o40_filter & gold_filter]

## If we were doing more analysis on this I'd want to look at some of these teams.... "White Lady", "Heroine", "Cincinnati Archers".

,ID,Age,Height,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
740,2735,41.0,168.000000,Russia,RUS,2000 Summer,2000,Summer,Sydney,Shooting,"Shooting Men's Rapid-Fire Pistol, 25 metres",Gold
1103,4029,46.0,175.908248,Argentina,ARG,1936 Summer,1936,Summer,Berlin,Polo,Polo Men's Polo,Gold
1421,4993,41.0,180.000000,Mosk II,NOR,1920 Summer,1920,Summer,Antwerpen,Sailing,Sailing Mixed 10 metres,Gold
3040,10769,49.0,180.000000,White Lady,DEN,1964 Summer,1964,Summer,Tokyo,Sailing,Sailing Mixed Three Person Keelboat,Gold
3270,11617,44.0,180.008757,Heroine,GBR,1908 Summer,1908,Summer,London,Sailing,Sailing Mixed 7 metres,Gold
...,...,...,...,...,...,...,...,...,...,...,...,...
38916,130271,43.0,170.500000,Heira II,NOR,1920 Summer,1920,Summer,Antwerpen,Sailing,Sailing Mixed 12 metres,Gold
38952,130381,42.0,167.000000,United States,USA,1972 Summer,1972,Summer,Munich,Archery,Archery Women's Individual,Gold
39193,130936,56.0,180.008757,United States,USA,1908 Summer,1908,Summer,London,Shooting,"Shooting Men's Running Target, Double Shot",Gold
39197,130936,60.0,178.673158,United States,USA,1912 Summer,1912,Summer,Stockholm,Art Competitions,Art Competitions Mixed Sculpturing,Gold


# Determine the Average Height of:

## (3A) All rows included.
- The average height of all rows is 5 feet and 9 inches

In [12]:
avg_height = (oly_df['Height'].mean()) * 0.39370
feet = int(avg_height // 12)
inches = int(avg_height % 12)

print(f"The average height of all rows is {feet} feet and {inches} inches")

The average height of all rows is 5 feet and 9 inches


## (3B) All rows included, but grouped by events.


In [13]:
round(oly_df.groupby(['Event'])['Height'].mean() * 0.39370, 2)

Event
Alpine Skiing Men's Combined                      69.67
Alpine Skiing Men's Downhill                      69.45
Alpine Skiing Men's Giant Slalom                  69.30
Alpine Skiing Men's Slalom                        69.23
Alpine Skiing Men's Super G                       70.83
                                                  ...  
Wrestling Women's Flyweight, Freestyle            62.24
Wrestling Women's Heavyweight, Freestyle          68.29
Wrestling Women's Light-Heavyweight, Freestyle    66.93
Wrestling Women's Lightweight, Freestyle          64.04
Wrestling Women's Middleweight, Freestyle         64.78
Name: Height, Length: 726, dtype: float64

## (3C) This dataset contains many athletes more than once (if they competed in multiple events and/or multiple years). 
- Average height is still 5 feet 9 inches.

In [ ]:
one_per_athlete = oly_df.drop_duplicates(subset = 'ID', keep = 'first')

avg_height2 = (one_per_athlete['Height'].mean()) * 0.39370
feet2 = int(avg_height2 // 12)
inches2 = int(avg_height2 % 12)

print(f"The average height of all rows is {feet2} feet and {inches2} inches")

The average height of all rows is 5 feet and 9 inches
